In [3]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


In [5]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

#Define a new graph
workflow = StateGraph(state_schema=MessagesState)


#Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    # Update message history with response:
    return {"messages": response}


#Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

#Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [6]:
config = {"configurable": {"thread_id": "test_1"}}

In [ ]:
system_prompt = """You are a friendly inquisitive chatbot. Your goal is to learn about the user by asking questions naturally.  
Gather the following details (prioritize missing info):  
- Name  
- Profession  
- Hobbies  
- Favorite books/movies  
- Life goals  

**Rules**:
1. Be conversational and friendly.
2. Ask one question at a time.
3. Only ask for missing information.
4. Never repeat questions.
5. Adapt based on their responses.
6. After each answer, summarize to yourself what you know so far."""

# Simplified prompt (no MessagesPlaceholder needed)
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    # User messages will be injected by LangGraph's state["messages"]
])